In [1]:
import numpy as np

In [2]:
from sympy import *

$U$ corresponds to a stretch/scale of the element.

In [3]:
def_gradient_no_rot_m = np.array([[3/2,0],[0,2]])

$$ {}^t_0\mathbf{\varepsilon}  = \frac{1}{2}(\mathbf{U}^2 - I) $$

In [4]:
strain_m = 0.5*(def_gradient_no_rot_m.dot(def_gradient_no_rot_m)-np.identity(2))
print(strain_m)

[[0.625 0.   ]
 [0.    1.5  ]]


Symmetric Matrix $$ {}^t_0\mathbf{\varepsilon}_{2x2} $$ to vector $$ {}^t_0\varepsilon_{rs} = [\varepsilon_{11},\varepsilon_{22},\varepsilon_{12}] $$

In [5]:
strain_v = np.array([strain_m[0][0],strain_m[1][1],strain_m[0][1]])
print(strain_v)

[0.625 1.5   0.   ]


In [6]:
E = 20000
v = 0.3

$$  \textbf{C} =
\frac{E}{(1+\nu)(1-2\nu)} \left(\begin{array}{cc} 
1-\nu & \nu & 0\\
\nu & 1-\nu & 0 \\
0 & 0 & \frac{1-2\nu}{2}
\end{array}\right)
$$ 

In [7]:
stiffness_m = E/((1+v)*(1-2*v))*np.array([[1-v,v,0],[v,1-v,0],[0,0,(1-2*v)/2]])
print(stiffness_m)

[[26923.07692308 11538.46153846     0.        ]
 [11538.46153846 26923.07692308     0.        ]
 [    0.             0.          7692.30769231]]


 $$ {}^t_0S_{ij} = C_{ijrs}{}^t_0\varepsilon_{rs} $$

In [8]:
stress_piola_v = stiffness_m.dot(strain_v) 
print(stress_piola_v)

[34134.61538462 47596.15384615     0.        ]


Interpolation functions of form: 
$$ h_1 = \frac{1}{4}(1+r)(1+s)$$
$h_{1,1}$ is the derivative w.r.t. $\frac{\partial}{\partial^0x_1}$ :
$$h_{1,1}=\frac{\partial h_1}{\partial r}\frac{\partial r}{\partial^0x_1} = \frac{1}{4}(1+s), \frac{\partial r}{\partial^0x_1} =1$$
resp.:
$$h_{1,2}=\frac{\partial h_1}{\partial s}\frac{\partial s}{\partial^0x_2} = \frac{1}{2}(1+r), \frac{\partial s}{\partial^0x_2} =2$$

In [9]:
x, y = symbols('x y')
scale=[1,1/2];
def h1(x,y): return 1/4*(1+x)*(1+y)
def h2(x,y): return 1/4*(1-x)*(1+y)
def h3(x,y): return 1/4*(1-x)*(1-y)
def h4(x,y): return 1/4*(1+x)*(1-y)
def hprime(h,x,y,j):
    dim =[x,y]
    return diff(h(x,y),dim[j]) / scale[j]
list_of_interp_fcts = np.array([h1,h2,h3,h4])

$$ {}^t_0\textbf{B}_{L0} = \begin{bmatrix}
h_{1,1} & 0 &|& ...\\
0 &h_{1,2} &|& ...\\
h_{1,2} & h_{1,1} &|& ...
\end{bmatrix} $$

In [10]:
BL = np.array([]).reshape(0,3)
for hi in list_of_interp_fcts:
    BL=np.concatenate((BL,np.array([
        [hprime(hi,x,y,0),0,hprime(hi,x,y,1)],
        [0,hprime(hi,x,y,1),hprime(hi,x,y,0)]
])))
BL =BL.T

In [11]:
xn = np.array([[2,1],[0,1],[0,0],[2,0]])
x0 = xn.T + np.array([np.repeat(1,1),np.repeat(1,1)])
R = np.array([[cos(np.radians(30)),-sin(np.radians(30))],[sin(np.radians(30)),cos(np.radians(30))]])


xt=R.dot(def_gradient_no_rot_m.dot(xn.T)) + np.array([np.repeat(4,4),np.repeat(1.5,4)])

In [12]:
u=xt-x0

$$l_{ij} = \sum_{k=1}^{N} h_{k,j} {}^tu_i^k$$

In [13]:
def lij(u,n,i,j):
    h=[h1,h2,h3,h4]
    lsum=0
    for k in range(n):
        lsum+=hprime(h[k],x,y,j)*u[i][k]
    return lsum

$$ {}^t_0\textbf{B}_{L1} = \begin{bmatrix}
l_{11}h_{1,1} & l_{21}h_{1,1} &|& ...\\
l_{12}h_{1,2} & l_{22}h_{1,2} &|& ...\\
l_{11}h_{1,2}+l_{12}h_{1,1} & l_{21}h_{1,2}+l_{22}h_{1,1} &|& ...
\end{bmatrix} $$

In [14]:
BL1 = np.array([]).reshape(0,3)
for hi in list_of_interp_fcts:
    BL1=np.concatenate((BL1,np.array([
        [hprime(hi,x,y,0)*lij(u,4,0,0),hprime(hi,x,y,1)*lij(u,4,0,1),hprime(hi,x,y,1)*lij(u,4,0,0)+hprime(hi,x,y,0)*lij(u,4,0,1)],
        [hprime(hi,x,y,0)*lij(u,4,1,0),hprime(hi,x,y,1)*lij(u,4,1,1),hprime(hi,x,y,1)*lij(u,4,1,0)+hprime(hi,x,y,0)*lij(u,4,1,1)]
])))
BL1 =BL1.T

$$ {}^t_0\textbf{B}_L = {}^t_0\textbf{B}_{L0} + {}^t_0\textbf{B}_{L1} $$

In [15]:
BL=BL+BL1

$$ J = \frac{\partial^0x_2}{\partial s} =\frac{1}{2} $$
$$ {}^t_0\textbf{F} = \int_{-1}^{1} \int_{-1}^{1} {}^t_0\textbf{B}^T_{L}{}_0^t\textbf{S}\frac{1}{2} dr ds $$

In [16]:
force_per_volume = BL.T.dot(stress_piola_v) *scale[0]*scale[1]


In [17]:
nodal_forces = np.empty(8);
for counter, element in enumerate(force_per_volume):
    nodal_forces[counter] = (N((integrate(integrate(element,(x,-1,1)),(y,-1,1)))))

Reshaping ${}^t_0\textbf{F}$ once to rotate, reshaping for the sake of staying with the convention.

In [18]:
nodal_forces = nodal_forces.reshape(4,2).T
print(nodal_forces)

[[-25425.07079254 -69767.23689977  25425.07079254  69767.23689977]
 [ 95239.43747563  69638.47593717 -95239.43747563 -69638.47593717]]


In [19]:
Rotate_cc_90 = np.array([[cos(np.radians(90)),-sin(np.radians(90))],[sin(np.radians(90)),cos(np.radians(90))]])

In [20]:
Rotate_cc_90.dot(nodal_forces).T.reshape(8)

array([-95239.4374756340, -25425.0707925378, -69638.4759371725,
       -69767.2368997699, 95239.4374756341, 25425.0707925378,
       69638.4759371725, 69767.2368997699], dtype=object)